In [5]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [6]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [7]:
fapeal = pd.DataFrame()

links_proyectos = []
fechas_proyectos = []
titulos_proyectos = []
pdfs_proyectos = []

estados = ["https://www.fapeal.br/category/editais/editais-vigentes/", "https://www.fapeal.br/category/editais/editais-encerrados/", "https://www.fapeal.br/category/editais/chamadas-abertas/"]


for estado in estados:



    #paginaciṕm
    parser_estado = parser(estado)
    paginacion = parser_estado.xpath('//div[@class="nav-links"]//text()')
    pag = []
    for i in paginacion:
        if i.isdigit():
            pag.append(i)
    
    pag_max = sorted(pag, reverse=True)[0]

    n = 1
    while n <= int(pag_max):

        end = "page/"+ str(n) +"/"

        if n == 1:
            parser_proyectos = parser(estado)
        else:
            parser_proyectos = parser(estado+end)

        titulos = []
        fechas = []
        links = []

        #link
        links = sorted(list(set(parser_proyectos.xpath('//div[@class="list-left-wrap"]//a/@href'))),reverse=True)

        #Titulos
        tits = parser_proyectos.xpath('//div[@class="list-left-wrap"]//h2[@class="entry-title"]/a/text()')
        for tit in tits:
            titulos.append(tit.strip())

        #Fechas
        fechas = parser_proyectos.xpath('//div[@class="list-left-wrap"]//div[@class="entry-meta"]/span/text()')

        for fecha in fechas:
            if fecha.isdigit():
                fechas.remove(fecha)
        
        n+=1

        links_proyectos+=links
        fechas_proyectos+=fechas
        titulos_proyectos+=titulos

# pdfs
for link in links_proyectos:

    parser_proyecto = parser(link)

    pdfs_proyecto = ''

    pdfs = parser_proyecto.xpath('//a/@href')

    for pdf in pdfs:
        if pdf.endswith('.pdf'):
            pdfs_proyecto = pdfs_proyecto +', '+ pdf

    pdfs_proyectos.append(pdfs_proyecto.strip(', '))

fapeal['Titulo'] = titulos_proyectos
fapeal['Fecha'] = fechas_proyectos
fapeal['link'] = links_proyectos
fapeal['pdfs'] = pdfs_proyectos

fapeal

,Titulo,Fecha,link,pdfs
0,Edital Fapeal/Fapesp 05/2019 – Auxílio à Pesqu...,22 de julho de 2020,https://www.fapeal.br/2020/07/edital-fapeal-fa...,http://www.fapeal.br/wp-content/uploads/2020/0...
1,Edital Fapeal/Confap/FBMG 04/2020 – Grand Cha...,7 de abril de 2020,https://www.fapeal.br/2020/04/edital-fapeal-co...,http://www.fapeal.br/wp-content/uploads/2020/0...
2,CONFAP: Chamada Emergencial da União Europeia ...,25 de março de 2020,https://www.fapeal.br/2020/03/confap-chamada-e...,http://www.fapeal.br/wp-content/uploads/2020/0...
3,Edital Confap Maeci14/2019,5 de dezembro de 2019,https://www.fapeal.br/2019/12/edital-confap-ma...,
4,Edital interno para os servidores concursados ...,19 de junho de 2019,https://www.fapeal.br/2019/06/edital-interno-p...,http://www.fapeal.br/wp-content/uploads/2019/0...
...,...,...,...,...
70,Edital Confap Maeci14/2019,5 de dezembro de 2019,https://www.fapeal.br/2019/12/edital-confap-ma...,
71,Edital Fapeal/Confap 06/2019 – Mobilidade Conf...,27 de novembro de 2019,https://www.fapeal.br/2019/11/edital-fapeal-co...,http://www.fapeal.br/wp-content/uploads/2019/0...
72,Edital Fapeal/Confap Nº8/2019 – BiodivERsA:,25 de setembro de 2019,https://www.fapeal.br/2019/09/edital-fapeal-co...,
73,Editais SibratecNANO – 6º Ciclo,2 de setembro de 2019,https://www.fapeal.br/2019/09/editais-sibratec...,


In [23]:
fapeal.to_excel('brasil_Fapeal.xlsx')

In [14]:
fapeal['link'][3]

'https://www.fapeal.br/2019/12/edital-confap-maeci14-2019/'